In [ ]:
!pip3 install html5lib bs4

In [ ]:
import os
from bs4 import BeautifulSoup

from settings import downloadsFolder, platformsOrdered

In [ ]:
filenames = sorted([name for name in os.listdir(downloadsFolder) 
                     if name.split("_")[0] in platformsOrdered])

In [ ]:
for i, name in enumerate(filenames[37:]):
    platform, rest = name.split("_")
    game = rest.replace(".html", "")
    urlpath = "/game/%s/%s" % (platform, game)
    print (i, platform, game, end =": ") # urlpath, end=" ")

    with open(os.path.join(downloadsFolder, name), "r") as f:
        page = f.read()
    # print(page)
    soup = BeautifulSoup(page, 'html5lib')
    # print(soup.prettify())
    
    ms = soup.find('div', attrs = {'class':'score_summary metascore_summary'})
    metascoreFind = ms.find('span', attrs = {'itemprop':'ratingValue'})
    # print (metascoreFind)
    metascore = int(metascoreFind.text) if metascoreFind else -1
    summary = ms.find('div', attrs = {'class' : 'summary'})
    criticReviews = summary.find('a', attrs = {'href':'%s/critic-reviews' % urlpath})
    metascoreBased = int(criticReviews.find('span').text.strip()) if criticReviews else 0
    print ("ms=%d (%d revs)" % (metascore, metascoreBased), end=" ")

    us = soup.find('div', attrs = {'class':'userscore_wrap feature_userscore'})
    # print (us.prettify())
    userscoreTags = us.select("div[class^=metascore_w\ user\ large\ game]")
    print(userscoreTags)
    if len(userscoreTags) !=1: raise Error()
    userscoreText = userscoreTags[0].text.strip()
    userscore = -1 if userscoreText=="tbd" else float(userscoreText)
    usersummary = us.find('div', attrs = {'class' : 'summary'})
    userReviews = usersummary.find('a', attrs = {'href':'%s/user-reviews' % urlpath})


    
    if userReviews:
        userscoreBased = int(userReviews.text.replace("Ratings","").strip())
    else:
        um = usersummary.find('span', attrs = {'class':'connect4_msg'}).text.strip()
        userscoreBased = -int(um.replace("Awaiting","").replace("more rating","").replace("s",""))
    print ("us=%.1f (%d revs)" % (userscore, userscoreBased), end=" ")

    textlines = [lines.strip() for lines in soup.body.text.split("\n") 
                    if lines.strip() != ""]
    try:
        nopsIndex = textlines.index("# of players:")
        nops = textlines[nopsIndex+1] if nopsIndex else ""
    except:
        nops = ""

    developer = textlines[textlines.index("Developer:")+1]
    print ("Dev=%s, #plyrs=%s" % (developer, nops)) # .prettify())
